In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import scipy
from scipy import special

# Encoprating errors into each cos sim measurment


In [2]:
raw_data = pd.read_csv('ctamdb_data_fcs_NTERA2.csv', sep=',') #imports the data into a pandas dataframe
raw_data = raw_data.T
raw_data.columns = raw_data.iloc[0]
raw_data = raw_data.drop(raw_data.index[0])
raw_data = raw_data.replace('absent',np.nan).dropna(axis='columns')
clean_data = raw_data.replace('no_change',np.nan).dropna(axis='columns')

In [3]:
raw_pvals = pd.read_csv('ctamdb_data_pvals_NTERA2.csv', sep=',') #imports the data into a pandas dataframe
raw_pvals = raw_pvals.T
raw_pvals.columns = raw_pvals.iloc[0]
raw_pvals = raw_pvals.drop(raw_pvals.index[0])
raw_pvals = raw_pvals.replace('absent',np.nan).dropna(axis='columns')
raw_pvals = raw_pvals.replace('no_change',np.nan).dropna(axis='columns')
raw_pvals =raw_pvals.astype('float')

In [4]:
raw_pvals.shape,clean_data.shape

((61, 13414), (61, 13414))

### prop of error for cos sim\

$$ \mathbf {a} \cdot \mathbf {b} =\|\mathbf {a} \|\ \|\mathbf {b} \|\cos \theta .$$

Rearanging gives 
$$ \cos \theta =  \frac {\mathbf {a} \cdot \mathbf {b}}{\|\mathbf {a} \|\ \|\mathbf {b} \|}  $$

error using prop of error:

$$ \sum_i \frac{1}{a^2b^2} [ a_i - \frac{\mathbf{a} \cdot \mathbf{b}}{b^2} ]^2 (err b_i)^2 + \sum_j \frac{1}{a^2b^2}[ b_j - \frac{\mathbf{a} \cdot \mathbf{b}}{a^2} ]^2 (err a_j)^2  $$

In [5]:
def sing_error(a,b,aerr,berr):
    
    def Merr(A,B,Berr):
        front_frac = 1/(np.dot(A,A)*np.dot(B,B))
        middle = (A - np.dot(A,B)/np.dot(B,B) )*Berr
        middle = (np.sum(middle**2))
        total = front_frac*middle
        return total
    
    term1 = Merr(a,b,berr)
    term2 = Merr(b,a,aerr)
    return (term1 + term2)**0.5   

def multi_error(a,B,aerr,Berr):#find the error between site a and B where B is a number of sites B i.e B = (b1,b2,b3...)
    front_frac1 = 1/(np.diag(np.dot(B.T,B))*np.dot(a,a))
    
    middel1 =np.stack((a,)*len(B.T) ,axis = 0)
    end =  (np.stack( (np.dot(a,B),)*len(a)).T )/np.stack( ((np.diag(np.dot(B.T,B))),)*len(a)  ).T 
    ME = ((middel1 - end).T *Berr)**2
    term1 = front_frac1*np.sum(ME)
    
    middel2 =( (B - (np.dot(a,B))/np.dot(a,a))* np.stack( (aerr,)*len(B.T) ).T )**2
    term2 = front_frac1*np.sum(middel2)
    return (term1 + term2)**0.5


def run_errorCS(FC_data,std_data):
    ErrorList = []
    for i,site in enumerate(FC_data.columns):
        print(i/len(FC_data.columns))
        a = FC_data[site]
        B = FC_data
        aerr = std_data[site]
        Berr = std_data
        propError = multi_error(a,B,aerr,Berr)
        ErrorList.append(propError)
    return ErrorList

In [6]:
#func to find the cosine similarity  vectors a and maxtrix B 
#where B is made up of vectors b1 b2... that are the same dim as a
#shape of B needs to be (a.shape,number of vectors to compare with)
import numpy.linalg as LA
def multi_cos_sim(a,B):
    #first to do some checks to see inputs are the right shape
    B_shape = B.shape 
    if B_shape[0] == len(a):
        numerator = np.dot(a,B)
        B_norm = LA.norm(B, axis=0)
        a_norm = LA.norm(a)
        return (numerator)/(B_norm*a_norm) # dont take abs yet intreased in postive and negative cosine sim
    else:
        print('Error dim of vectors in B, ' + str(B_shape[0]) + ' do not match dim a ' + str(len(a)))
        pass

def main_cos_sim(dataframe): #method to find the cosine similairty between all pairs of sites in a clean phos dataframe   
    simList = []
    i = 0 
    #for loop to apply above method to data
    for c in dataframe.columns:
        #print(i,c)
        v1 = dataframe[c].to_numpy()
        V2 = dataframe.to_numpy() #turns the table to numpy array, taking out lables
        V2 = V2 #turns data in V2 to np.float64 data type, required to use LA.norm func
        cosSim = multi_cos_sim(v1,V2)
        simList.append((cosSim))
        i = i +1
    return simList

In [7]:
import time
def cluster_no_duplicates(Correlated_sites,CS_DF,CS_threshold): #clustering were only one site can be in one cluster 
    t1 = time.perf_counter()
    used_site = []
    clust = []
    unique_site = np.unique(Correlated_sites[0])
    # CS_threshold sets threshold for a site to be within a cluster 
    #loop over all unique sites
    i = 0
    for s in unique_site:
        if s in used_site:
            pass
        else:
            clust.append([s] ) #start a cluster with s  
            #consider if the sites that are highly correlated with s will be in the cluster 
            considered_sites = (Correlated_sites[s == Correlated_sites[0]][1]).values
            CS_DF_subset = CS_DF[clust[i]]
            for s2 in considered_sites:
                s2_clust_CS = CS_DF_subset.loc[s2] #finds the CS between s2 all sites in the cluster under consideration
                #checks if s2 is within the threshold to all other sites currnetly in the cluster
                num_thresh = np.sum(s2_clust_CS > CS_threshold) 
                #if s2 is closer than the threshold to all sites in the cluster under consideration
                #add it to the cluster and remove it from the sites that need to be condiered in the future 
                #also remove it from the correlated_sites list so that its not used again in another cluster 
                if num_thresh == len(s2_clust_CS) and s2 not in used_site:
                    clust[i].append(s2)
                    used_site.append(s2)
                    #CS_DF = CS_DF.drop(s2)
                    #CS_DF = CS_DF.drop(columns = s2)
            i = i + 1 
            used_site.append(s2)
            #CS_DF = CS_DF.drop(s)
            #CS_DF = CS_DF.drop(columns = s)   
    print(time.perf_counter() - t1)
    return clust

## Try squashing data with the error funciton 
this means the data will then be scalled between -1 and 1 

### Look at setting 'down' to -1 , 'up' to 1 and 'no change' to zero after applying erf 

In [8]:
#values the data needs to be set to before the erf is applied  
special.erf(10),special.erf(-10),special.erf(0)

(1.0, -1.0, 0.0)

In [3]:
#need to find the sites that share idenical FC vales with other sites
#create  removeData df to do this
removeData = raw_data.replace('absent',np.nan).dropna(axis='columns')
removeData = removeData.replace('no_change',np.nan).dropna(axis='columns')
removeData = removeData.replace('down',np.nan)
removeData = removeData.replace('up',np.nan)
#need to fill the 'up' 'down' with random vales so they don't come up in the anylsis of finding 
# mistaken identical sites
M = len(removeData.index)
N = len(removeData.columns)
ran = pd.DataFrame(np.random.randn(M,N), columns=removeData.columns, index=removeData.index)
#updates ran with the removeData DF, apart from the nan's in removeData
#these nans are the values we want to put random data in 
ran.update(removeData) 

def removeErrSitesRaw(df): #take out sits that share exact values with another site
    del_sitesL = []
    cols = list(df.columns)
    for i,s in enumerate(cols): #loop over all cols in df 
        vs = df[s]
        print(i)
        #finds sites to compare to vs, (taking out s) so we don't compare vs site with itself
        comp_sites = cols[0:i] + cols[i+1:len(cols)] 
        comp = df[comp_sites]
        mask = (comp.T == vs)
        del_sites = mask.index[np.sum(mask,axis = 1) > 0] #sites with 1 or more identical FC values to s
        if len(del_sites)>0:
            del_sitesL.extend(del_sites) 
    return del_sitesL
del_sitesL = removeErrSitesRaw(ran)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700


5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340


6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980


8421
8422
8423
8424
8425
8426
8427
8428
8429
8430
8431
8432
8433
8434
8435
8436
8437
8438
8439
8440
8441
8442
8443
8444
8445
8446
8447
8448
8449
8450
8451
8452
8453
8454
8455
8456
8457
8458
8459
8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620


10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10127
10128
10129
10130
10131
10132
10133
10134
10135
10136
10137
10138
10139
10140
10141
10142
10143
10144
10145
10146
10147
10148
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10180
10181
10182
10183
10184
10185
10186
10187
10188
10189
10190
10191
10192
10193
10194
10195
10196
10197
10198
10199
10200
10201
10202
10203
10204
10205
10206
10207
10208
10209
10210
10211
10212
10213
10214
10215
10216
1021

11417
11418
11419
11420
11421
11422
11423
11424
11425
11426
11427
11428
11429
11430
11431
11432
11433
11434
11435
11436
11437
11438
11439
11440
11441
11442
11443
11444
11445
11446
11447
11448
11449
11450
11451
11452
11453
11454
11455
11456
11457
11458
11459
11460
11461
11462
11463
11464
11465
11466
11467
11468
11469
11470
11471
11472
11473
11474
11475
11476
11477
11478
11479
11480
11481
11482
11483
11484
11485
11486
11487
11488
11489
11490
11491
11492
11493
11494
11495
11496
11497
11498
11499
11500
11501
11502
11503
11504
11505
11506
11507
11508
11509
11510
11511
11512
11513
11514
11515
11516
11517
11518
11519
11520
11521
11522
11523
11524
11525
11526
11527
11528
11529
11530
11531
11532
11533
11534
11535
11536
11537
11538
11539
11540
11541
11542
11543
11544
11545
11546
11547
11548
11549
11550
11551
11552
11553
11554
11555
11556
11557
11558
11559
11560
11561
11562
11563
11564
11565
11566
11567
11568
11569
11570
11571
11572
11573
11574
11575
11576
11577
11578
11579
11580
11581
11582
1158

12783
12784
12785
12786
12787
12788
12789
12790
12791
12792
12793
12794
12795
12796
12797
12798
12799
12800
12801
12802
12803
12804
12805
12806
12807
12808
12809
12810
12811
12812
12813
12814
12815
12816
12817
12818
12819
12820
12821
12822
12823
12824
12825
12826
12827
12828
12829
12830
12831
12832
12833
12834
12835
12836
12837
12838
12839
12840
12841
12842
12843
12844
12845
12846
12847
12848
12849
12850
12851
12852
12853
12854
12855
12856
12857
12858
12859
12860
12861
12862
12863
12864
12865
12866
12867
12868
12869
12870
12871
12872
12873
12874
12875
12876
12877
12878
12879
12880
12881
12882
12883
12884
12885
12886
12887
12888
12889
12890
12891
12892
12893
12894
12895
12896
12897
12898
12899
12900
12901
12902
12903
12904
12905
12906
12907
12908
12909
12910
12911
12912
12913
12914
12915
12916
12917
12918
12919
12920
12921
12922
12923
12924
12925
12926
12927
12928
12929
12930
12931
12932
12933
12934
12935
12936
12937
12938
12939
12940
12941
12942
12943
12944
12945
12946
12947
12948
1294

np.savetxt('NTERA2_remove_sites',np.unique(del_sitesL),fmt ='%s')

In [10]:
clean_data = clean_data.replace('down',-10)
clean_data = clean_data.replace('up',10)
clean_data = clean_data.replace('no_change',0)
clean_data = clean_data.astype(float)
clean_data.shape
squashed_data = special.erf(clean_data)
squashed_data.shape

(61, 13414)

In [18]:
squashed_data = special.erf(clean_data)
#find cos sim for above data 
PairSimList = main_cos_sim(squashed_data)
CS_squash_inf = pd.DataFrame(PairSimList,index = squashed_data.columns, columns= squashed_data.columns)
np.fill_diagonal(CS_squash_inf.values,0) #fills diag with 0 i.e. the sites comapred with themself
CS_squash_inf.head() #DF gives the cos_sim between rows and column, note diag 1 as it compares the same sites

pst,AAAS(S495),AAGAB(S310),AAGAB(S311),AAK1(S14),AAK1(S21),AAK1(S624),AAK1(S637),AAK1(S678),AAK1(T389),AAK1(T606),...,ZYX(S259),ZYX(S267),ZYX(S281),ZYX(S308),ZYX(S344),ZYX(T270),ZYX(T274),ZZEF1(S1518),ZZZ3(S113),ZZZ3(S82)
pst,,,,,,,,,,,,,,,,,,,,,
AAAS(S495),0.000000,0.226382,0.476832,0.330655,0.200380,-0.169832,0.243248,-0.313432,0.340458,-0.420563,...,-0.262690,-0.376492,-0.237513,-0.090666,0.075585,-0.268249,0.021886,0.201926,0.579296,-0.043247
AAGAB(S310),0.226382,0.000000,0.700171,0.044007,0.047623,0.150014,-0.107252,0.041537,0.141867,-0.289431,...,0.007147,-0.133626,0.006970,0.031881,0.053712,0.083193,0.020806,0.001636,0.205653,0.065123
AAGAB(S311),0.476832,0.700171,0.000000,0.123152,-0.013789,0.051060,-0.047800,-0.224964,0.307795,-0.258297,...,-0.047451,-0.060067,0.066098,0.029157,0.157467,0.091095,0.150483,0.130721,0.368356,-0.005607
AAK1(S14),0.330655,0.044007,0.123152,0.000000,0.706804,0.092454,0.180015,-0.264603,0.365074,-0.259380,...,0.139618,0.094265,0.046035,0.164666,0.049729,-0.108224,-0.044074,0.365739,0.103147,-0.016502
AAK1(S21),0.200380,0.047623,-0.013789,0.706804,0.000000,0.048656,0.135022,-0.228807,0.256726,-0.028908,...,0.417071,0.156756,0.138019,0.097688,-0.005367,-0.126952,-0.016414,0.077727,0.052531,0.152502


In [19]:
del PairSimList

In [61]:
cos98temp = CS_squash_inf[CS_squash_inf > 0.98].dropna(how = 'all')

#taking the NaNs out of the above list and pairing the correlated sites with each other
FL = []
for i,r in cos98temp.iterrows():
    #print(i)
    #print('****')
    #print(r.dropna())
    for ind,val in zip(r.dropna().index,r.dropna()):
        FL.append( (i,ind,val) )
cosSim98_squash_inf = pd.DataFrame(FL)
#cosSim98fliped = pd.DataFrame({0: cosSim98[1],1: cosSim98[0],2:cosSim98[2] })
cosSim98_squash_inf 

,0,1,2
0,AAK1(S624),AAK1(T620),1.000000
1,AAK1(T620),AAK1(S624),1.000000
2,AATF(S320),AATF(S321),1.000000
3,AATF(S321),AATF(S320),1.000000
4,ABI2(S183),RILPL2(S107),0.999359
...,...,...,...
2491,ZNF768(S97),ZNF768(S90),1.000000
2492,ZPBP(S24),GOLGA4(S1248),0.981025
2493,ZPBP(S24),PKLR(S485),0.997406
2494,ZSWIM9(S420),TRIM46(S92),0.994739


Shows that taking into account the inifins and -infins about doubles the amount of highly correlated pairs of sites. 

## propogage p vals though the error func 
then apply propoation through the cos sim 

In [12]:
raw_pvals.shape,clean_data.shape

((61, 13414), (61, 13414))

In [13]:
stdDF = raw_pvals*clean_data
stdDF

pst,AAAS(S495),AAGAB(S310),AAGAB(S311),AAK1(S14),AAK1(S21),AAK1(S624),AAK1(S637),AAK1(S678),AAK1(T389),AAK1(T606),...,ZYX(S259),ZYX(S267),ZYX(S281),ZYX(S308),ZYX(S344),ZYX(T270),ZYX(T274),ZZEF1(S1518),ZZZ3(S113),ZZZ3(S82)
AC220,0.006001,-0.055055,-0.090830,-0.098699,0.014837,1.387274e-02,-0.097481,0.048875,-0.070575,0.039718,...,0.026165,-0.035938,-0.050535,-3.099340e-02,-3.067007e-02,-0.037632,-0.183350,-0.030194,0.069420,0.065995
AT13148,-0.040321,0.079171,0.134683,-0.088760,-0.049842,3.782338e-02,-0.000657,0.004731,-0.071745,0.037244,...,-0.007721,0.031625,-0.016307,3.061036e-02,3.089111e-02,0.072987,-0.159750,0.091792,-0.039093,-0.053139
AZ20,-0.040939,-2.481139,-0.186268,0.041516,-0.073383,-2.537652e-03,-0.055841,0.000638,-0.008834,0.044044,...,0.032310,0.005147,0.019185,8.480641e-05,1.992039e-03,0.002014,0.013525,-0.091731,-0.075654,-0.053099
AZD1480,-0.033960,-0.107863,0.033480,0.028474,0.031071,5.687746e-14,-0.053126,-0.074431,0.120911,0.027046,...,0.018798,0.025479,-0.015108,-1.156535e-07,1.582177e-07,0.000037,0.095951,-0.079339,-0.051075,0.059909
AZD3759,-0.024981,-0.120157,-0.120157,-0.054610,-0.062808,-1.595897e-03,-0.012084,-0.080566,-0.080126,0.038167,...,0.036572,0.030285,0.007732,-3.819324e-02,-2.216752e-02,0.064390,0.013963,-0.093534,-0.061454,-0.065812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Torin,-0.025213,0.071670,0.061473,0.069562,0.019039,4.143029e-02,0.020125,-0.003001,0.094162,-0.047485,...,-0.016939,0.028772,0.008504,2.311502e-04,3.197152e-02,0.071982,0.175474,-0.123216,-0.058380,-0.043791
Trametinib,-0.042836,-0.095470,-0.089184,0.041006,0.068735,4.177945e-02,-0.056409,-0.002095,-0.066152,0.010960,...,-0.006224,-0.001676,0.036562,3.282285e-02,-1.005416e-02,0.035681,0.089297,-0.090926,-0.040578,-0.015264
U73122,0.020341,0.055220,0.000029,0.058019,0.067243,-2.619299e-03,-0.060557,-0.002507,-0.091290,-0.023191,...,-0.001843,-0.023007,0.010971,-3.588620e-02,1.731540e-02,-0.071118,0.010176,-0.082327,0.062314,0.038915
Ulixertinib,0.023675,0.063398,0.079555,0.004333,0.058298,-1.961619e-03,-0.027969,-0.003952,0.055020,-0.027403,...,-0.039451,-0.034031,-0.037400,-3.456719e-02,2.017218e-02,-0.037928,0.055819,0.068857,0.020761,0.041492


In [14]:
stdDF_erf = stdDF*np.exp(-clean_data**2)*(2/ (np.pi**0.5) )
stdDF_erf

pst,AAAS(S495),AAGAB(S310),AAGAB(S311),AAK1(S14),AAK1(S21),AAK1(S624),AAK1(S637),AAK1(S678),AAK1(T389),AAK1(T606),...,ZYX(S259),ZYX(S267),ZYX(S281),ZYX(S308),ZYX(S344),ZYX(T270),ZYX(T274),ZZEF1(S1518),ZZZ3(S113),ZZZ3(S82)
AC220,0.006771,-6.107692e-02,-9.541565e-02,-0.089137,0.016725,1.563892e-02,-9.293254e-02,0.054324,-0.076731,0.044318,...,0.029381,-0.039434,-0.054165,-3.471304e-02,-3.243210e-02,-0.002085,-0.081304,-0.001261,0.074827,0.069664
AT13148,-0.044506,1.936643e-02,6.962462e-02,-0.089987,-0.032841,4.209651e-02,-7.223177e-07,0.000733,-0.060007,0.041622,...,-0.006691,0.034255,-0.018376,3.427179e-02,3.357103e-02,0.065707,-0.129056,0.075925,-0.043719,-0.058591
AZ20,-0.044035,-1.041497e-43,-3.334084e-02,0.046396,-0.077614,-2.863357e-03,-6.171916e-02,0.000719,-0.009965,0.048888,...,0.034628,0.004782,0.018151,5.378131e-05,1.756805e-03,0.000516,0.001062,-0.096162,-0.077817,-0.058550
AZD1480,-0.037980,-1.065076e-01,3.754316e-02,0.031998,0.034879,9.073876e-17,-5.891883e-02,-0.056899,0.101672,0.030390,...,0.019017,0.028599,-0.017029,-4.091756e-08,6.015780e-08,0.000001,0.089374,-0.069958,-0.056516,0.065282
AZD3759,-0.028083,-1.181488e-01,-1.181488e-01,-0.023644,-0.068828,-9.079058e-04,-1.362703e-02,-0.048140,-0.081656,0.042634,...,0.040628,0.033906,0.008722,-4.240653e-02,-2.493775e-02,0.071176,0.000148,-0.098553,-0.067579,-0.071158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Torin,-0.028342,7.741891e-02,6.742835e-02,0.057718,0.021447,4.472886e-02,2.266508e-02,-0.000317,0.079672,-0.052482,...,-0.019084,0.031055,0.007991,1.680542e-04,3.512549e-02,0.007946,0.129816,-0.116756,-0.057491,-0.048786
Trametinib,-0.047355,-8.134995e-02,-7.890305e-02,0.026380,0.070250,4.586193e-02,-4.586935e-02,-0.000166,-0.072604,0.012361,...,-0.007021,-0.001891,0.040235,3.546037e-02,-9.591087e-03,0.003129,0.013498,-0.028778,-0.045274,-0.017205
U73122,0.019507,2.557275e-02,2.190445e-06,0.045629,0.070851,-1.381344e-03,-3.823038e-02,-0.000002,-0.097210,-0.026096,...,-0.002079,-0.025869,0.012372,-3.990189e-02,1.950553e-02,-0.027004,0.000084,-0.083881,0.059994,0.043500
Ulixertinib,0.023153,3.234399e-02,8.298727e-02,0.001355,0.054702,-9.640996e-04,-3.143679e-02,-0.000009,0.021292,-0.030794,...,-0.043520,-0.037052,-0.038952,-3.666087e-02,2.270591e-02,-0.005168,0.003427,0.063913,0.014108,0.046298


In [15]:
FC_data = squashed_data
std_data = stdDF_erf
CS_errEfcT = run_errorCS(FC_data,std_data)

0.0
7.45489786789921e-05
0.0001490979573579842
0.0002236469360369763
0.0002981959147159684
0.0003727448933949605
0.0004472938720739526
0.0005218428507529447
0.0005963918294319368
0.0006709408081109288
0.000745489786789921
0.0008200387654689131
0.0008945877441479052
0.0009691367228268973
0.0010436857015058893
0.0011182346801848814
0.0011927836588638736
0.0012673326375428656
0.0013418816162218577
0.00141643059490085
0.001490979573579842
0.001565528552258834
0.0016400775309378262
0.0017146265096168183
0.0017891754882958103
0.0018637244669748023
0.0019382734456537946
0.002012822424332787
0.0020873714030117787
0.002161920381690771
0.0022364693603697627
0.002311018339048755
0.0023855673177277472
0.002460116296406739
0.0025346652750857313
0.0026092142537647235
0.0026837632324437154
0.0027583122111227076
0.0028328611898017
0.0029074101684806917
0.002981959147159684
0.003056508125838676
0.003131057104517668
0.0032056060831966602
0.0032801550618756525
0.0033547040405546443
0.0034292530192336365


0.02937229759952289
0.029446846578201877
0.02952139555688087
0.029595944535559863
0.029670493514238856
0.029745042492917848
0.029819591471596837
0.02989414045027583
0.029968689428954823
0.030043238407633815
0.030117787386312808
0.0301923363649918
0.03026688534367079
0.030341434322349783
0.030415983301028775
0.030490532279707768
0.03056508125838676
0.030639630237065753
0.030714179215744743
0.030788728194423735
0.030863277173102728
0.03093782615178172
0.031012375130460713
0.031086924109139706
0.031161473087818695
0.031236022066497688
0.031310571045176684
0.03138512002385567
0.03145966900253466
0.031534217981213655
0.03160876695989265
0.03168331593857164
0.03175786491725063
0.031832413895929626
0.03190696287460862
0.03198151185328761
0.032056060831966604
0.0321306098106456
0.03220515878932459
0.032279707768003575
0.03235425674668257
0.03242880572536156
0.03250335470404055
0.032577903682719546
0.03265245266139854
0.03272700164007753
0.032801550618756524
0.03287609959743552
0.03295064857611

0.05956463396451469
0.059639182943193675
0.05971373192187267
0.05978828090055166
0.05986282987923065
0.059937378857909646
0.06001192783658864
0.06008647681526763
0.060161025793946624
0.060235574772625616
0.06031012375130461
0.0603846727299836
0.060459221708662594
0.06053377068734158
0.06060831966602057
0.060682868644699565
0.06075741762337856
0.06083196660205755
0.060906515580736544
0.060981064559415536
0.06105561353809453
0.06113016251677352
0.061204711495452514
0.06127926047413151
0.0613538094528105
0.061428358431489485
0.06150290741016848
0.06157745638884747
0.06165200536752646
0.061726554346205456
0.06180110332488445
0.06187565230356344
0.061950201282242434
0.06202475026092143
0.06209929923960042
0.06217384821827941
0.062248397196958405
0.06232294617563739
0.06239749515431638
0.062472044132995376
0.06254659311167438
0.06262114209035337
0.06269569106903236
0.06277024004771134
0.06284478902639033
0.06291933800506933
0.06299388698374832
0.06306843596242731
0.0631429849411063
0.0632175

0.09027881318025943
0.09035336215893842
0.09042791113761742
0.09050246011629641
0.0905770090949754
0.0906515580736544
0.09072610705233339
0.09080065603101238
0.09087520500969137
0.09094975398837037
0.09102430296704934
0.09109885194572834
0.09117340092440733
0.09124794990308632
0.09132249888176532
0.09139704786044431
0.0914715968391233
0.0915461458178023
0.09162069479648129
0.09169524377516028
0.09176979275383927
0.09184434173251826
0.09191889071119726
0.09199343968987625
0.09206798866855524
0.09214253764723424
0.09221708662591323
0.09229163560459222
0.09236618458327121
0.0924407335619502
0.0925152825406292
0.09258983151930819
0.09266438049798718
0.09273892947666618
0.09281347845534516
0.09288802743402415
0.09296257641270314
0.09303712539138213
0.09311167437006113
0.09318622334874012
0.09326077232741911
0.0933353213060981
0.0934098702847771
0.09348441926345609
0.09355896824213508
0.09363351722081407
0.09370806619949307
0.09378261517817206
0.09385716415685105
0.09393171313553005
0.094006

0.12106754137468316
0.12114209035336215
0.12121663933204115
0.12129118831072014
0.12136573728939913
0.12144028626807812
0.12151483524675712
0.12158938422543611
0.1216639332041151
0.1217384821827941
0.12181303116147309
0.12188758014015208
0.12196212911883107
0.12203667809751007
0.12211122707618906
0.12218577605486805
0.12226032503354704
0.12233487401222604
0.12240942299090503
0.12248397196958402
0.12255852094826301
0.122633069926942
0.122707618905621
0.12278216788429999
0.12285671686297897
0.12293126584165796
0.12300581482033696
0.12308036379901595
0.12315491277769494
0.12322946175637393
0.12330401073505293
0.12337855971373192
0.12345310869241091
0.1235276576710899
0.1236022066497689
0.12367675562844789
0.12375130460712688
0.12382585358580588
0.12390040256448487
0.12397495154316386
0.12404950052184285
0.12412404950052185
0.12419859847920084
0.12427314745787983
0.12434769643655882
0.12442224541523782
0.12449679439391681
0.12457134337259579
0.12464589235127478
0.12472044132995377
0.124794

...

0.8536603548531385
0.8537349038318175
0.8538094528104965
0.8538840017891755
0.8539585507678544
0.8540330997465335
0.8541076487252125
0.8541821977038915
0.8542567466825705
0.8543312956612494
0.8544058446399284
0.8544803936186074
0.8545549425972864
0.8546294915759655
0.8547040405546444
0.8547785895333234
0.8548531385120024
0.8549276874906814
0.8550022364693604
0.8550767854480393
0.8551513344267183
0.8552258834053973
0.8553004323840764
0.8553749813627554
0.8554495303414343
0.8555240793201133
0.8555986282987923
0.8556731772774713
0.8557477262561503
0.8558222752348292
0.8558968242135083
0.8559713731921873
0.8560459221708663
0.8561204711495453
0.8561950201282242
0.8562695691069032
0.8563441180855822
0.8564186670642612
0.8564932160429403
0.8565677650216192
0.8566423140002982
0.8567168629789772
0.8567914119576562
0.8568659609363352
0.8569405099150141
0.8570150588936931
0.8570896078723721
0.8571641568510512
0.8572387058297302
0.8573132548084091
0.8573878037870881
0.8574623527657671
0.8575369017

0.8860146115998211
0.8860891605785001
0.886163709557179
0.886238258535858
0.886312807514537
0.8863873564932161
0.8864619054718951
0.886536454450574
0.886611003429253
0.886685552407932
0.886760101386611
0.88683465036529
0.886909199343969
0.8869837483226479
0.887058297301327
0.887132846280006
0.887207395258685
0.887281944237364
0.8873564932160429
0.8874310421947219
0.8875055911734009
0.8875801401520799
0.887654689130759
0.8877292381094379
0.8878037870881169
0.8878783360667959
0.8879528850454749
0.8880274340241538
0.8881019830028328
0.8881765319815118
0.8882510809601909
0.8883256299388699
0.8884001789175489
0.8884747278962278
0.8885492768749068
0.8886238258535858
0.8886983748322648
0.8887729238109437
0.8888474727896227
0.8889220217683018
0.8889965707469808
0.8890711197256598
0.8891456687043388
0.8892202176830177
0.8892947666616967
0.8893693156403757
0.8894438646190547
0.8895184135977338
0.8895929625764127
0.8896675115550917
0.8897420605337707
0.8898166095124497
0.8898911584911287
0.889965

0.9184434173251826
0.9185179663038616
0.9185925152825406
0.9186670642612196
0.9187416132398986
0.9188161622185776
0.9188907111972566
0.9189652601759356
0.9190398091546146
0.9191143581332936
0.9191889071119725
0.9192634560906515
0.9193380050693305
0.9194125540480096
0.9194871030266886
0.9195616520053675
0.9196362009840465
0.9197107499627255
0.9197852989414045
0.9198598479200835
0.9199343968987624
0.9200089458774415
0.9200834948561205
0.9201580438347995
0.9202325928134785
0.9203071417921574
0.9203816907708364
0.9204562397495154
0.9205307887281944
0.9206053377068734
0.9206798866855525
0.9207544356642314
0.9208289846429104
0.9209035336215894
0.9209780826002684
0.9210526315789473
0.9211271805576263
0.9212017295363053
0.9212762785149844
0.9213508274936634
0.9214253764723424
0.9214999254510213
0.9215744744297003
0.9216490234083793
0.9217235723870583
0.9217981213657372
0.9218726703444163
0.9219472193230953
0.9220217683017743
0.9220963172804533
0.9221708662591322
0.9222454152378112
0.9223199642

0.9507976740718652
0.9508722230505442
0.9509467720292232
0.9510213210079022
0.9510958699865811
0.9511704189652602
0.9512449679439392
0.9513195169226182
0.9513940659012972
0.9514686148799761
0.9515431638586551
0.9516177128373341
0.9516922618160131
0.9517668107946922
0.9518413597733711
0.9519159087520501
0.9519904577307291
0.9520650067094081
0.9521395556880871
0.952214104666766
0.952288653645445
0.952363202624124
0.9524377516028031
0.9525123005814821
0.952586849560161
0.95266139853884
0.952735947517519
0.952810496496198
0.952885045474877
0.9529595944535559
0.953034143432235
0.953108692410914
0.953183241389593
0.953257790368272
0.9533323393469509
0.9534068883256299
0.9534814373043089
0.9535559862829879
0.953630535261667
0.953705084240346
0.9537796332190249
0.9538541821977039
0.9539287311763829
0.9540032801550619
0.9540778291337408
0.9541523781124198
0.9542269270910988
0.9543014760697779
0.9543760250484569
0.9544505740271358
0.9545251230058148
0.9545996719844938
0.9546742209631728
0.954748

0.9832264797972268
0.9833010287759057
0.9833755777545847
0.9834501267332637
0.9835246757119428
0.9835992246906218
0.9836737736693008
0.9837483226479797
0.9838228716266587
0.9838974206053377
0.9839719695840167
0.9840465185626956
0.9841210675413746
0.9841956165200537
0.9842701654987327
0.9843447144774117
0.9844192634560907
0.9844938124347696
0.9845683614134486
0.9846429103921276
0.9847174593708066
0.9847920083494857
0.9848665573281646
0.9849411063068436
0.9850156552855226
0.9850902042642016
0.9851647532428806
0.9852393022215595
0.9853138512002385
0.9853884001789176
0.9854629491575966
0.9855374981362756
0.9856120471149545
0.9856865960936335
0.9857611450723125
0.9858356940509915
0.9859102430296705
0.9859847920083494
0.9860593409870285
0.9861338899657075
0.9862084389443865
0.9862829879230655
0.9863575369017444
0.9864320858804234
0.9865066348591024
0.9865811838377814
0.9866557328164605
0.9867302817951394
0.9868048307738184
0.9868793797524974
0.9869539287311764
0.9870284777098554
0.9871030266

In [ ]:
CS_errorEfc = pd.DataFrame(CS_errEfcT)
CS_errorEfc.index = CS_errorEfc.columns
CS_errorEfc.to_csv('CS_error_Erf_NTERA2.csv')

In [17]:
CS_errorEfcDF = pd.read_csv('CS_error_Erf_NTERA2.csv', sep=',') 
CS_errorEfcDF.drop(columns = 'pst',inplace = True)
CS_errorEfcDF.index = CS_errorEfcDF.columns
CS_errorEfcDF.head()                                

,AAAS(S495),AAGAB(S310),AAGAB(S311),AAK1(S14),AAK1(S21),AAK1(S624),AAK1(S637),AAK1(S678),AAK1(T389),AAK1(T606),...,ZYX(S259),ZYX(S267),ZYX(S281),ZYX(S308),ZYX(S344),ZYX(T270),ZYX(T274),ZZEF1(S1518),ZZZ3(S113),ZZZ3(S82)
AAAS(S495),0.082466,0.023765,0.033831,0.034509,0.030760,0.016883,0.030233,0.021641,0.032237,0.084541,...,0.022436,0.042786,0.029294,0.017693,0.016341,0.026410,0.018542,0.029244,0.058612,0.033183
AAGAB(S310),0.023765,0.036272,0.026040,0.020777,0.021553,0.017149,0.021124,0.016436,0.022039,0.069253,...,0.016352,0.026836,0.022322,0.017516,0.017990,0.017295,0.019035,0.022459,0.029728,0.031214
AAGAB(S311),0.033831,0.026040,0.036559,0.021511,0.020735,0.015262,0.021288,0.017108,0.024673,0.062804,...,0.016529,0.021766,0.020547,0.013652,0.016116,0.017088,0.017939,0.023933,0.036208,0.029286
AAK1(S14),0.034509,0.020777,0.021511,0.048869,0.039351,0.016555,0.027149,0.018120,0.032451,0.052522,...,0.019590,0.025893,0.025038,0.017947,0.015876,0.018254,0.017188,0.031986,0.030558,0.032189
AAK1(S21),0.030760,0.021553,0.020735,0.039351,0.050083,0.018746,0.024354,0.018736,0.028389,0.034252,...,0.026893,0.029605,0.027635,0.015805,0.015036,0.018720,0.020001,0.023156,0.030192,0.043070


## cos sim with erros 

In [21]:
def Find_sim_sites(CS_data,err_df,threshold):
    cos98temp = CS_data[CS_data - err_df*2 > threshold].dropna(how = 'all')
    #taking the NaNs out of the above list and pairing the correlated sites with each other
    FL = []
    for i,r in cos98temp.iterrows():
        #print(i)
        #print('****')
        #print(r.dropna())
        for ind,val in zip(r.dropna().index,r.dropna()):
            FL.append( (i,ind,val) )
    cosSim98_err_inf = pd.DataFrame(FL)
    #cosSim98fliped = pd.DataFrame({0: cosSim98_err_inf[1],1: cosSim98_err_inf[0],2:cosSim98_err_inf[2] })
    return cosSim98_err_inf
cosSim98_err_inf = Find_sim_sites(CS_squash_inf,CS_errorEfcDF,0.98)
cosSim98_err_inf

,0,1,2
0,BDP1(S883),KBTBD7(S15),1.0
1,BDP1(S883),ZC3H3(S329),1.0
2,C18orf63(S398),C18orf63(S400),1.0
3,C18orf63(S398),FBXO39(S310),1.0
4,C18orf63(S398),FBXO39(S312),1.0
...,...,...,...
321,ZC3H3(S329),KBTBD7(S15),1.0
322,ZMYM5(S336),CLSPN(T493),1.0
323,ZMYM5(S336),FPGT(S24),1.0
324,ZMYND8(S707),ZMYND8(S709),1.0


In [22]:
CS_threshold = 0.98
Full_erf_clust = cluster_no_duplicates(cosSim98_err_inf,CS_squash_inf,CS_threshold)
Full_erf_clust = sorted(Full_erf_clust, key = len,reverse=True) #sorts list from most clusters to least
Full_erf_clust

0.052949000004446134


[['C18orf63(S398)',
  'C18orf63(S400)',
  'FBXO39(S310)',
  'FBXO39(S312)',
  'GRIP1(S981)',
  'GRIP1(T983)',
  'IGSF10(S2537)',
  'IGSF10(T2540)',
  'LRTM2(S168)',
  'LRTM2(S170)',
  'RIDA(S2)',
  'RIDA(S3)',
  'STYXL2(S965)',
  'STYXL2(S966)',
  'TPP2(S221)',
  'TPP2(T222)'],
 ['SRSF2(S206)', 'SRSF2(S208)', 'SRSF2(S212)', 'SRSF2(S221)'],
 ['BDP1(S883)', 'KBTBD7(S15)', 'ZC3H3(S329)'],
 ['CLSPN(T493)', 'FPGT(S24)', 'ZMYM5(S336)'],
 ['CTCF(S609)', 'CTCF(S610)', 'CTCF(S612)'],
 ['HIRIP3(S330)', 'HIRIP3(S332)', 'HIRIP3(S333)'],
 ['NCL(S28)', 'NCL(S34)', 'NCL(S41)'],
 ['CUTC(S17)', 'OTOGL(S1571)'],
 ['DNAJC1(S479)', 'DNAJC1(S480)'],
 ['HMGA1(S102)', 'HMGA1(S103)'],
 ['IWS1(S248)', 'IWS1(S250)'],
 ['MLLT6(S378)', 'MLLT6(S382)'],
 ['NCL(S184)', 'NCL(S206)'],
 ['NFATC2IP(S90)', 'NFATC2IP(S92)'],
 ['NOL8(S890)', 'NOL8(T888)'],
 ['OSBP2(S284)', 'OSBP2(S287)'],
 ['PDAP1(S60)', 'PDAP1(S63)'],
 ['RFC1(S283)', 'RFC1(Y282)'],
 ['SRRM2(S1913)', 'SRRM2(S1916)'],
 ['SRRM2(S808)', 'SRRM2(T810)'],
 ['SRS

In [23]:
for clust in erf_clust:
    for i in range(0,len(clust)):
        try:
            diff = np.sum(squashed_data[clust[i]] - squashed_data[clust[i +1]] )
            print(diff)
            print('')
        except IndexError:
            pass
    print('-----')


NameError: name 'erf_clust' is not defined